In [1]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import glob
import cv2
import locale
import os
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

### Load in Shankar array Data

In [4]:
shankarrs = []

with open('data-sa.csv') as f:
    for line in f.readlines():
        shankarrs.append(eval(line))

shankarrs = np.array(shankarrs)
np.save('shankarrs-simple', shankarrs)

In [5]:
shankarrs = np.load('shankarrs-simple.npy')

### Load in player hand data

In [6]:
player_hands = []

with open('data-ph.csv') as f:
    for line in f.readlines():
        player_hands.append(eval(line))

player_hands = np.array([arr.flatten() for arr in np.array(player_hands)])
np.save('player-hands-simple', player_hands)

### transition to superpermutation

In [7]:
shankarrs_sp = np.array([np.array([e[0],e[1],e[2],e[3],e[0],e[1],e[2],e[0],e[3],e[1],e[2],e[0],e[1],e[3],e[2],e[0],e[1]]) for e in shankarrs])

### Switch shankar arrays to categorical representation

In [8]:
shankarrs_cr = np.zeros((len(shankarrs_sp), 17, 13, 3))
for i in range(len(shankarrs_sp)):
    for j in range(17):
        for k in range(13):
            l = shankarrs_sp[i][j][k]
            if l > 0:
                shankarrs_cr[i][j][k][l - 1] = 1

In [18]:
np.save('shankarrs-cr-simple', shankarrs_cr)

### From np file

In [2]:
shankarrs_cr = np.load('shankarrs-cr-simple.npy')
player_hands = np.load('player-hands-simple.npy')/10

### split train and test data

In [3]:
i = int(len(shankarrs_cr) / 2)
j = int(len(shankarrs_cr) * 3 / 4)

trainX = shankarrs_cr[:i]
testX = shankarrs_cr[i:j]
endX = shankarrs_cr[j:]

trainY = player_hands[:i]
testY = player_hands[i:j]
endY = player_hands[j:]

### Create CNN model

In [4]:
def build_model(hp):
    inputs = Input(shape=(17, 13, 3))

    x = inputs
    
    for i in range(3):
    
        x = Conv2D(filters = hp.Int(f'conv_{i + 1}_filter', min_value=32, max_value=128, step=16),
                   kernel_size = hp.Choice(f'conv_{i + 1}_kernel', values = [3,5]),
                   strides = (1, 1),
                   padding = "same"
                  )(x)
        x = BatchNormalization(axis=-1, scale = False)(x)
        x = Activation("relu")(x)
#         x = MaxPooling2D(pool_size=(3, 3))(x)

    x = BatchNormalization(axis=-1, scale = False)(x)

    x = Flatten()(x)

    for i in range(3):
        x = Dense(hp.Int(f'dense_{i+1}_units', min_value=32, max_value=128, step=16))(x)
        x = Activation("sigmoid")(x)

    x = Dense(52)(x)
    x = Activation("softmax")(x)

    # x = BatchNormalization(axis=-1)(x)
    # x = Dropout(0.5)(x)

    model = Model(inputs, x)

    opt = Adam(
           learning_rate = hp.Choice('learning_rate', values=[1.0, 0.1, 0.01, 0.001]),      # 0.001
           beta_1=0.9,                                                                      # 0.9
           beta_2=0.999,                                                                    # 0.999
           epsilon=0.1,                                                                     # 1e-7
           amsgrad=False,                                                                   # False
           name="Adam"
          )

    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy','mean_squared_error', 'cosine_similarity', 'logcosh'])
    
    return model

### Compile

### random search for best parameters

In [21]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=1,directory='output',project_name="gin")

INFO:tensorflow:Reloading Oracle from existing project output/gin/oracle.json


In [22]:
tuner_search.search(trainX,trainY,validation_data=(testX, testY), epochs=1, batch_size = 1000, verbose=True)

Epoch 1/3
1614/1614 [==============================] - ETA: 0s - loss: 4.1569 - accuracy: 0.0000e+00 - mean_squared_error: 0.0017 - cosine_similarity: 0.3614 - logcosh: 8.5130e-0 - ETA: 10:38 - loss: 4.1559 - accuracy: 0.0000e+00 - mean_squared_error: 0.0017 - cosine_similarity: 0.3617 - logcosh: 8.5102e- - ETA: 14:04 - loss: 4.1589 - accuracy: 0.0000e+00 - mean_squared_error: 0.0017 - cosine_similarity: 0.3609 - logcosh: 8.5180e- - ETA: 15:51 - loss: 4.1568 - accuracy: 0.0000e+00 - mean_squared_error: 0.0017 - cosine_similarity: 0.3616 - logcosh: 8.5105e- - ETA: 16:46 - loss: 4.1545 - accuracy: 0.0000e+00 - mean_squared_error: 0.0017 - cosine_similarity: 0.3625 - logcosh: 8.5022e- - ETA: 17:24 - loss: 4.1538 - accuracy: 0.0000e+00 - mean_squared_error: 0.0017 - cosine_similarity: 0.3629 - logcosh: 8.4984e- - ETA: 17:51 - loss: 4.1517 - accuracy: 1.4286e-04 - mean_squared_error: 0.0017 - cosine_similarity: 0.3636 - logcosh: 8.4914e- - ETA: 18:11 - loss: 4.1501 - accuracy: 1.2500e-04 - 

INFO:tensorflow:Oracle triggered exit


In [23]:
cnn = tuner_search.get_best_models(num_models=1)[0]

In [24]:
cnn.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 17, 13, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 17, 13, 96)        2688      
_________________________________________________________________
batch_normalization (BatchNo (None, 17, 13, 96)        288       
_________________________________________________________________
activation (Activation)      (None, 17, 13, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 13, 48)        41520     
_________________________________________________________________
batch_normalization_1 (Batch (None, 17, 13, 48)        144       
_________________________________________________________________
activation_1 (Activation)    (None, 17, 13, 48)        0     

In [25]:
cnn.save('tuned-1.h5')

### Train

In [28]:
cnn.fit(
        trainX, trainY,
        validation_data=(testX, testY),
        epochs=3, batch_size=25
)

Epoch 1/3
 6559/64545 [==>...........................] - ETA: 0s - loss: 3.7721 - accuracy: 0.0000e+00 - mean_squared_error: 0.0014 - cosine_similarity: 0.4972 - logcosh: 7.2221e-0 - ETA: 26:09 - loss: 3.7798 - accuracy: 0.0133 - mean_squared_error: 0.0015 - cosine_similarity: 0.4937 - logcosh: 7.2549e-0 - ETA: 31:44 - loss: 3.7774 - accuracy: 0.0160 - mean_squared_error: 0.0014 - cosine_similarity: 0.4958 - logcosh: 7.2311e- - ETA: 32:51 - loss: 3.7803 - accuracy: 0.0171 - mean_squared_error: 0.0014 - cosine_similarity: 0.4949 - logcosh: 7.2414e- - ETA: 33:25 - loss: 3.7718 - accuracy: 0.0133 - mean_squared_error: 0.0014 - cosine_similarity: 0.4971 - logcosh: 7.2200e- - ETA: 33:28 - loss: 3.7816 - accuracy: 0.0109 - mean_squared_error: 0.0015 - cosine_similarity: 0.4942 - logcosh: 7.2482e- - ETA: 33:18 - loss: 3.7792 - accuracy: 0.0092 - mean_squared_error: 0.0014 - cosine_similarity: 0.4948 - logcosh: 7.2432e- - ETA: 33:19 - loss: 3.7799 - accuracy: 0.0107 - mean_squared_error: 0.001

KeyboardInterrupt: 

### End - Testing

In [38]:
def ten_largest(arr):
    new_arr = [*arr]
    for i in range(15):
        new_arr[new_arr.index(max(new_arr))] = -1
    return [1 if new_arr[i] == -1 else 0 for i in range(52)]

In [39]:
t = 0

X = endX
Y = endY
P = cnn.predict(np.array([X[t]]))[0]


np.set_printoptions(suppress=True)

print("test case:", t)
# print("stage:", sum(list(X[t][i]).count(-1) + list(X[t][i]).count(1) for i in range(4)))
print()
print("Shankararr:")
print(X[t])
print()
print("Prediction:")
print(P)
print()
print("  pred:"," ".join('{:.2f}'.format(round(e,2)) for e in P))
print("actual:", " ".join(f"0.{int(e * 10)} " for e in Y[t]))
print("\n", "-" * 150, "\n")
print("    rounded:", *[int(round(p * 10)) for p in P])
print("ten largest:", *ten_largest(P))
print("     actual:", *[int(e * 10) for e in Y[t]])
print()
print("mean squared diff:", sum((P - Y[t]) ** 2) / 52)

test case: 0

Shankararr:
[[[0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 0. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]

 [[0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [1. 0. 0.]
  [0. 1. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]

 [[0. 1. 0.]
  [0. 0. 0.]
  [0. 1. 0.]
  [0. 0. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [1. 0. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [1. 0. 0.]]

 [[0. 0. 0.]
  [0. 1. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]

 [[0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 0. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]

 [[0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  

In [37]:
eval(str(P)[1:-1].replace(" ", "*").replace("\n",''))

4.211949604732173e-91

In [14]:
cnn.save('sankarr-simple.h5')

# Extra test

In [53]:
sa = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], [0, -2, 0, 0, -2, 0, 0, 0, 0, 0, -2, -2, -1], [0, -2, 0, -2, 0, 0, 0, 0, -2, -2, 0, -1, -1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, -2]]
sa17 = np.array([sa[0],sa[1],sa[2],sa[3],sa[0],sa[1],sa[2],sa[0],sa[3],sa[1],sa[2],sa[0],sa[1],sa[3],sa[2],sa[0],sa[1]])
ph = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0], [0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]]
phf = np.array(ph).flatten()/10

In [54]:
print("stage:", sum(list(sa[i]).count(-1) + list(sa[i]).count(1) for i in range(4)))
pred = cnn.predict(np.array([sa17]))[0]
print()
print(sa17)
print()
print(pred)
print()
np.set_printoptions(suppress=True)
print("    rounded:", *[int(round(p * 10)) for p in pred])
print("ten largest:", *ten_largest(pred))
print("     actual:", *[int(e * 10) for e in phf])
print()
print("loss:", sum((pred - phf) ** 2) / 52)

stage: 4

[[ 0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [ 0 -2  0  0 -2  0  0  0  0  0 -2 -2 -1]
 [ 0 -2  0 -2  0  0  0  0 -2 -2  0 -1 -1]
 [ 0  0  0  0  0  0  0  0  0  0  0 -2 -2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [ 0 -2  0  0 -2  0  0  0  0  0 -2 -2 -1]
 [ 0 -2  0 -2  0  0  0  0 -2 -2  0 -1 -1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [ 0  0  0  0  0  0  0  0  0  0  0 -2 -2]
 [ 0 -2  0  0 -2  0  0  0  0  0 -2 -2 -1]
 [ 0 -2  0 -2  0  0  0  0 -2 -2  0 -1 -1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [ 0 -2  0  0 -2  0  0  0  0  0 -2 -2 -1]
 [ 0  0  0  0  0  0  0  0  0  0  0 -2 -2]
 [ 0 -2  0 -2  0  0  0  0 -2 -2  0 -1 -1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [ 0 -2  0  0 -2  0  0  0  0  0 -2 -2 -1]]

[0.00000455 0.00000593 0.00002898 0.00000001 0.00014803 0.00000001
 0.00020389 0.00000001 0.         0.00032353 0.00000118 0.00048993
 0.00618647 0.00091192 0.08792629 0.00000043 0.00000001 0.09555292
 0.         0.00232864 0.00000001 0.00353669 0.00836828 0.06561217
 0.097